Saves member daily checkin
* Time data is in string but no need to save is in corresponding type to the db
* Date data however should

In [1]:
import os
import sys
import pandas as pd

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from src.utils.process_raw_data import filter_raw_log_data_remake

In [2]:
link_raw = "data/Original Records Report_JAN2023_à_JUIN 2023.csv"
link_filter = parent_dir + "/src/utils/data/personnel_ARTI_2023_no_directors.csv"

In [3]:
# Retruns raw filtered data, and the filter df
logs_raw, data_filter = filter_raw_log_data_remake(link_raw, link_filter) 

In [4]:
def in_out(x):
    if x[-1:]=="1": return "IN"
    if x[-1:]=="2": return "OUT"

logs_raw["Gate"] = logs_raw['Attendance Check Point'].map(lambda x: in_out(x))
logs_raw["Date"] = logs_raw.Time.map(lambda x: x.split(" ")[0])
logs_raw["Hour"] = logs_raw.Time.map(lambda x: x.split(" ")[1])

df = logs_raw[["matricule","base","Date","Hour","Gate",]].copy()
df.dropna(inplace=True)
df.rename(columns={"matricule":"log_member_id","base":"log_member_name",
                   "Date":"log_date","Hour":"log_time","Gate":"log_gate"}, inplace=True)
df["log_date"] = pd.to_datetime(df["log_date"])

In [5]:
df.head(1)

,log_member_id,log_member_name,log_date,log_time,log_gate
259,1390722B,ERIC BAILLY THEODORE,2023-03-10,11:26:03,OUT


Insert data into mongodb

In [6]:
from src.config.db import nlogsrawdb
nlogsrawdb.insert_many(df.to_dict(orient="records"))